In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [7]:
pd.read_sql("""
SELECT
  p.subject_id,
  p.external_id,
  p.full_name,
  a.hadm_id,
  a.admission_type,
  a.admittime,
  a.dischtime,
  a.hospital_expire_flag
FROM patients p
JOIN admissions a ON a.subject_id = p.subject_id
ORDER BY a.admittime;
""", engine)

,subject_id,external_id,full_name,hadm_id,admission_type,admittime,dischtime,hospital_expire_flag
0,1,MRN-0001,Ana López,1,Emergency,2101-01-10 08:00:00,2101-01-15 14:00:00,False
1,6,MRN-0006,Lucía Herrera,7,Emergency,2101-02-05 09:30:00,2101-02-08 10:00:00,False
2,2,MRN-0002,Carlos Pérez,3,Emergency,2101-03-20 22:00:00,2101-03-28 10:00:00,True
3,7,MRN-0007,Miguel Castillo,9,Emergency,2101-05-14 07:00:00,2101-05-20 15:00:00,False
4,3,MRN-0003,María Gómez,4,Urgent,2101-07-11 13:00:00,2101-07-14 11:00:00,False
5,8,MRN-0008,Sofía Morales,10,Emergency,2101-08-21 20:00:00,2101-08-24 09:00:00,False
6,4,MRN-0004,José Martínez,5,Emergency,2101-09-02 06:00:00,2101-09-10 15:00:00,False
7,6,MRN-0006,Lucía Herrera,8,Urgent,2101-11-01 16:00:00,2101-11-04 12:00:00,False
8,5,MRN-0005,Alex Rivera,6,Emergency,2101-12-18 19:00:00,2101-12-22 08:00:00,False
9,1,MRN-0001,Ana López,2,Elective,2102-06-01 10:00:00,2102-06-05 09:00:00,False


In [10]:
pd.read_sql("""
SELECT
  p.external_id,
  p.full_name,
  COUNT(*) AS n_admissions
FROM patients p
JOIN admissions a ON a.subject_id = p.subject_id
GROUP BY p.external_id, p.full_name
HAVING COUNT(*) > 1
ORDER BY n_admissions DESC;
""", engine)

,external_id,full_name,n_admissions
0,MRN-0006,Lucía Herrera,2
1,MRN-0001,Ana López,2


In [12]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
FLOOR(
  EXTRACT(EPOCH FROM (a.dischtime - a.admittime)) / 86400
) AS length_of_stay_days

FROM admissions a
JOIN patients p ON p.subject_id = a.subject_id
ORDER BY length_of_stay_days DESC;
""", engine)

,hadm_id,external_id,full_name,length_of_stay_days
0,5,MRN-0004,José Martínez,8.0
1,3,MRN-0002,Carlos Pérez,7.0
2,9,MRN-0007,Miguel Castillo,6.0
3,1,MRN-0001,Ana López,5.0
4,6,MRN-0005,Alex Rivera,3.0
5,2,MRN-0001,Ana López,3.0
6,7,MRN-0006,Lucía Herrera,3.0
7,8,MRN-0006,Lucía Herrera,2.0
8,4,MRN-0003,María Gómez,2.0
9,10,MRN-0008,Sofía Morales,2.0


In [13]:
pd.read_sql("""
SELECT
a.hadm_id,
p.external_id,
p.full_name,
MAX(le.value_num) AS max_creatinine
FROM labevents le
JOIN d_labitems d ON d.labitem_id = le.labitem_id
JOIN admissions a ON a.hadm_id = le.hadm_id
JOIN patients p ON p.subject_id = a.subject_id
WHERE d.label = 'Creatinine'
GROUP BY a.hadm_id, p.external_id, p.full_name
ORDER BY max_creatinine DESC;
""",engine)

,hadm_id,external_id,full_name,max_creatinine
0,3,MRN-0002,Carlos Pérez,3.1
1,9,MRN-0007,Miguel Castillo,1.8
2,1,MRN-0001,Ana López,1.6
3,8,MRN-0006,Lucía Herrera,1.4
4,7,MRN-0006,Lucía Herrera,0.9


In [14]:
pd.read_sql("""
WITH labs AS (
  SELECT
    a.hadm_id,
    p.external_id,
    p.full_name,
    MAX(CASE WHEN d.label = 'White Blood Cells' THEN le.value_num END) AS max_wbc,
    MAX(CASE WHEN d.label = 'Lactate' THEN le.value_num END) AS max_lactate
  FROM labevents le
  JOIN d_labitems d ON d.labitem_id = le.labitem_id
  JOIN admissions a ON a.hadm_id = le.hadm_id
  JOIN patients p ON p.subject_id = a.subject_id
  WHERE d.label IN ('White Blood Cells', 'Lactate')
  GROUP BY a.hadm_id, p.external_id, p.full_name
)
SELECT *
FROM labs
WHERE max_wbc > 12 OR max_lactate > 2
ORDER BY hadm_id;
""", engine)

,hadm_id,external_id,full_name,max_wbc,max_lactate
0,3,MRN-0002,Carlos Pérez,18.4,3.8
1,5,MRN-0004,José Martínez,14.2,2.2
2,6,MRN-0005,Alex Rivera,12.5,NaN
3,10,MRN-0008,Sofía Morales,13.0,NaN
